In [ ]:
import os
import pandas as pd
import numpy as np
import sklearn.tree as tr
from sklearn import preprocessing
path_to_dataset_folder = 'C:/Users/Dimiexe/Desktop/DATAsets'
target_test_path = path_to_dataset_folder + '/DATA/Target_feature_test_CSVs'
tree_meta_feature_path = path_to_dataset_folder + '/DATA/tree_metafeatures_for_test_CSVs'

In [ ]:
#Used to make a dictionary with leave nodes' ids maped to their branch lengths. So there are that many key:value
#entries as the numbeer of leaves. The aformentioned dictionary is the global variable "branch_lengths".
branch_lengths = {}
def get_branches_len(input_tree_, start_node, counter):
    if input_tree_.children_left[start_node] !=-1:
        counter += 1
        get_branches_len(input_tree_,input_tree_.children_left[start_node],counter)
        get_branches_len(input_tree_,input_tree_.children_right[start_node],counter)
    else:
        global branch_lengths
        branch_lengths[start_node] = counter

In [ ]:
#Returns a list of ints, containing the node ids of the leaves.
def find_leaves(input_tree_):
    leaves = []
    for node in range(input_tree_.node_count):
        if input_tree_.children_left[node] == -1:
            leaves.append(node)
    return leaves

In [ ]:
#Returns a dictionary with level-id int keys maped to the nodes on that level. e.g. 0:0, 1:[1,2] for a simple 2-level tree
#root + 2 children nodes.
def get_levels(input_tree_):
    levels = {0:[0]}
    for i in range(input_tree_.max_depth):
        left_children = [input_tree_.children_left[x] for x in levels[i] if input_tree_.children_left[x] != -1]
        right_children = [input_tree_.children_right[x] for x in levels[i] if input_tree_.children_right[x] != -1]
        levels[i+1] = left_children + right_children
    return levels

In [ ]:
#Returns a dictionary with the features as keys, and their frequency of appearance as values.
def get_feature_freq(input_tree_):
    freqs = {}
    for i in range(input_tree_.node_count):
        if input_tree_.children_left[i] !=-1:
            if input_tree_.feature[i] in freqs:
                freqs[input_tree_.feature[i]] +=1
            else:
                freqs[input_tree_.feature[i]] = 1
    return freqs

In [ ]:
#The below code was found at: https://www.geeksforgeeks.org/diameter-of-a-binary-tree-in-on-a-new-method/
#and tweaked slightly to match the needs of this work.

# Function to find height of a tree  
def height(tree_,root, ans): 
    if (root == -1): 
        return 0
  
    left_height = height(tree_,tree_.children_left[root], ans)  
  
    right_height = height(tree_,tree_.children_right[root], ans)  
  
    # update the answer, because diameter  
    # of a tree is nothing but maximum  
    # value of (left_height + right_height + 1) 
    # for each node  
    ans[0] = max(ans[0], 1 + left_height + 
                             right_height)  
  
    return 1 + max(left_height, 
                   right_height) 
  
# Computes the diameter of binary  
# tree with given root.  
def diameter(tree_,root): 
    if (root < 0):  
        return 0
    ans = [-999999999999] # This will store 
                          # the final answer  
    height_of_tree = height(tree_,root, ans)  
    return ans[0]

In [ ]:
#Returns the 14-element feature vector (list) of the given dataFrame (df). Also if "visual_tree" is True,
#then the tree is ploted, default value: False.
def metafeatures(df, visual_tree = False):
    meta_vector = []
    x = df.loc[:,df.columns[0:-1]]
    y = df.loc[:,df.columns[-1]]
    regr = tr.DecisionTreeRegressor(random_state = 8328, min_impurity_decrease = 1e-06)
    regr.fit(x,y)
    if visual_tree:
        tr.plot_tree(regr)
    
    #1 Tree width (diameter)
    meta_vector.append(diameter(regr.tree_,0))    
    #2 Tree height
    meta_vector.append(regr.tree_.max_depth)    
    #3 Total number of nodes
    meta_vector.append(regr.tree_.node_count)    
    #4 Total number of leaves
    meta_vector.append(len(find_leaves(regr.tree_)))
    
    levels = get_levels(regr.tree_)
    nodes_per_level = [len(levels[i]) for i in range(len(levels))]    
    #5 Maximun nodes per level
    meta_vector.append(np.max(nodes_per_level))
    #6 Mean number of nodes per level
    meta_vector.append(np.mean(nodes_per_level))
    #7 Standard deviation of nodes per level
    meta_vector.append(np.std(nodes_per_level))
    
    global branch_lengths
    branch_lengths = {} #(re)initialise the global variable to hold the branches and their lengths
    get_branches_len(regr.tree_,0,0)
    length_per_branch = [branch_lengths[x] for x in branch_lengths]
    #(8) Longest branch's length| Not used because it's always the same as #2 Tree height
    #meta_vector.append(np.max(length_per_branch))
    #8 Shortest branch's length
    meta_vector.append(np.min(length_per_branch))
    #9 Mean length of branches
    meta_vector.append(np.mean(length_per_branch))
    #10 Standard deviation of length of branches
    meta_vector.append(np.std(length_per_branch))
    
    feature_frequencies = get_feature_freq(regr.tree_)
    freqs = [feature_frequencies[x] for x in feature_frequencies]
    #11 Maximum frequency of feature appearance
    meta_vector.append(np.max(freqs))
    #12 Minimum frequency of feature appearance
    meta_vector.append(np.min(freqs))
    #13 Mean frequency of feature appearance
    meta_vector.append(np.mean(freqs))
    #14 Standard deviation of frequency of feature appearance
    meta_vector.append(np.std(freqs))
    
    return meta_vector

In [ ]:
#This function prepares the dataframe for metafeature extraction. 
#First a missing values imputation is performed using the "method" arguement:
# drop = all instances with at least one missing value are droped,
# mean = missing values are assigned the mean value for each numerical feature,
# median = missing values are assigned the median value for each numerical feature.
#In any case, any missing values on categorical deatures are filled with the mode of
#each feature, the value most frequently seen.
#Returns the processed dataframe and the NaNs per line metric.
def tree_ready(df, method = 'drop'):
    total_nans = df.isna().sum().sum()
    mean_nans_per_line = total_nans / df.shape[0]
    feats_to_encode = [feat for i, feat in enumerate(df.columns) if df.dtypes[i] == 'object']
    if total_nans > 0:
        cols = np.setdiff1d(df.columns, feats_to_encode)
        if method == 'drop':
            df.dropna(axis = 0, how = 'any', inplace = True)
        elif method == 'mean':            
            df.fillna({col: df.loc[:,col].mean() for col in cols}, inplace = True)
            df.fillna({col: df.loc[:,col].mode()[0] for col in feats_to_encode}, inplace = True)
        elif method == 'median':
            df.fillna({col: df.loc[:,col].median() for col in cols}, inplace = True)
            df.fillna({col: df.loc[:,col].mode()[0] for col in feats_to_encode}, inplace = True)
        else:
            print('Non identifiable method provided!')
            return None, None
        
    for x in feats_to_encode:
        le = preprocessing.LabelEncoder()
        df.loc[:,x] = le.fit_transform(df.loc[:,x])
    return df , mean_nans_per_line

In [ ]:
#This creates and processes the tree to get the metafatures. The metafeature vector is saved in
#a txt file located in "../DATAsets/DATA/tree_metafeatures_for_test_CSVs" dir.
datasets = os.listdir(target_test_path)
extra_ignore = ['COMBO17%e.W462FE.csv', 'SkillCraft1_Dataset%WorkersMade.csv'] #These two only have 1 node in their trees, the root.
for file in np.sort(np.setdiff1d(datasets, extra_ignore, assume_unique=True)):
    print('proccessing file: ' + file)
    for method in ['drop','mean','median']:
        if not os.path.isfile(tree_meta_feature_path + '/' + method + '/' + file):
            print('method: ' + method)
            df = pd.read_table(target_test_path + '/'+ file,sep=',')
            df, MNpL = tree_ready(df, method)

            with open(tree_meta_feature_path + '/' + method + '/' + file,'w' ) as f:
                for i,x in enumerate(metafeatures(df)):
                    f.write(str(x)+',')
                f.write(str(MNpL))